<table align="left"><td>
  <a target="_blank"  href="https://colab.research.google.com/drive/1kAfRZS8HjWQjQyKqiJbot6tmTlwc550Z?usp=sharing">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab
  </a>
</td></table>

In [1]:
#libraries
from numpy import array

import tensorflow as tf

from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from numpy import asarray
from numpy import zeros
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers

import numpy as np
from pickle import dump
from pickle import load
import pandas as pd
import re


from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import classification_report



import tensorflow.keras.models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, SpatialDropout1D, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow import keras

In [2]:
#Importing the dataset
!gdown https://drive.google.com/u/0/uc?id=1x769lbHKYdLU9JMEo9V4g1-DbiOr4Ppi

Downloading...
From: https://drive.google.com/u/0/uc?id=1x769lbHKYdLU9JMEo9V4g1-DbiOr4Ppi
To: /content/dataset.csv
100% 356/356 [00:00<00:00, 710kB/s]


In [3]:
df = pd.read_csv('dataset.csv')
df = df.dropna()

df = df.astype({"Company_Info": int, "Activities": int, 'Others': int})
df

,Question,Company_Info,Activities,Others
0,What is your compliance?,1,0,0
1,What is the address of our Headquarters,1,0,0
2,What are the other Office Locations,1,0,0
3,Do you have any philanthropy commitments? If y...,1,1,0
4,Do we need to have a CRM solution to use Avnio?,1,0,1
5,Please provide products sold by your company,1,0,0


In [4]:
#splitting into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [5]:
#cleaning text
import re
def cleantext(text):
    nl = re.compile('([\n][\s]*)*')
    text = nl.sub('',text)
    return text

train['Question'] = train['Question'].apply(cleantext)
test['Question'] = test['Question'].apply(cleantext)
def max_len(x):
    a=x.split()
    return len(a)

max(train['Question'].apply(max_len))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


12

In [6]:
#tokenise text

vect=Tokenizer()
vect.fit_on_texts(train['Question'])
vocab_size = len(vect.word_index) + 1
print(vocab_size)

30


In [7]:
#getting the labels
labels = ['Company_Info',	'Activities',	'Others']
labels_df = train[labels]
y_train = labels_df.values
labels_df_test = test[labels]
y_test = labels_df_test.values

In [8]:
y_train

array([[1, 0, 1],
       [1, 1, 0],
       [1, 0, 0],
       [1, 0, 0]])

In [9]:
#encoding the padding text
encoded_docs_train = vect.texts_to_sequences(train['Question'])
encoded_docs_test = vect.texts_to_sequences(test['Question'])
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=50, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=50, padding='post')
print(padded_docs_train)

[[ 1  6  7  2  3  8  9 10  2 11 12  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 1 13  3 14 15 16 17 18 19 20 21 22  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 4  5 23 24  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]
 [ 4  5 25 26 27 28 29  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0]]


In [10]:
n_classes = 3

In [11]:
#I have designed a small seq-seq LSTM model for this purpose

model = Sequential()
# Configuring the parameters
model.add(Embedding(vocab_size, output_dim=50, input_length=50))
model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 50)            1500      
_________________________________________________________________
lstm (LSTM)                  (None, 50, 128)           91648     
_________________________________________________________________
dropout (Dropout)            (None, 50, 128)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 3)                 195       
Total params: 142,751
Trainable params: 142,751
Non-trainable params: 0
__________________________________________________

In [12]:
#training the model
from sklearn.utils import class_weight
model.compile(optimizer='adam', loss='binary_crossentropy')
history = model.fit(padded_docs_train, y_train,  epochs=5, batch_size=32, validation_split=0.1, callbacks=[])

Epoch 1/5
1/1 [==============================] - 1s 640ms/step - loss: 0.6961 - val_loss: 0.6856
Epoch 2/5
1/1 [==============================] - 0s 30ms/step - loss: 0.6867 - val_loss: 0.6735
Epoch 3/5
1/1 [==============================] - 0s 30ms/step - loss: 0.6743 - val_loss: 0.6592
Epoch 4/5
1/1 [==============================] - 0s 29ms/step - loss: 0.6650 - val_loss: 0.6425
Epoch 5/5
1/1 [==============================] - 0s 29ms/step - loss: 0.6508 - val_loss: 0.6208


In [13]:
#evaluation with different thresholds

#f1_score = 2 * (precision * recall) / (precision + recall)
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

def f1micro(y_true, y_pred):
    return tf.py_func(f1_score(y_true, y_pred,average='micro'),tf.double)

predictions=model.predict([padded_docs_test])
thresholds=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for val in thresholds:
    pred=predictions.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='micro')
    recall = recall_score(y_test, pred, average='micro')
    f1 = f1_score(y_test, pred, average='micro')
   
    print("Micro-average quality numbers for threshold: "+ str(val))
    print("Precision: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}".format(precision, recall, f1))

Micro-average quality numbers for threshold: 0.1
Precision: 0.3333, Recall: 1.0000, F1-Score: 0.5000
Micro-average quality numbers for threshold: 0.2
Precision: 0.3333, Recall: 1.0000, F1-Score: 0.5000
Micro-average quality numbers for threshold: 0.3
Precision: 0.3333, Recall: 1.0000, F1-Score: 0.5000
Micro-average quality numbers for threshold: 0.4
Precision: 0.3333, Recall: 1.0000, F1-Score: 0.5000
Micro-average quality numbers for threshold: 0.5
Precision: 1.0000, Recall: 1.0000, F1-Score: 1.0000
Micro-average quality numbers for threshold: 0.6
Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
Micro-average quality numbers for threshold: 0.7
Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
Micro-average quality numbers for threshold: 0.8
Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000
Micro-average quality numbers for threshold: 0.9
Precision: 0.0000, Recall: 0.0000, F1-Score: 0.0000


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
